# 02 – Build Historical AQS Data for Oregon (2020–2024)

This notebook fetches daily air quality data for Oregon from the EPA AQS API
and saves it to local parquet files in the `data/` folder.

It pulls:

- PM2.5 (parameter code `88101`)
- Ozone (parameter code `44201`)

for years 2020–2024, for Oregon (`state` FIPS code `"41"`).

Credentials (`AQS_EMAIL` and `AQS_KEY`) are read from environment variables so
that no secrets are stored in the notebook or repository.


In [ ]:
import os
import time
from datetime import date
from pathlib import Path

import pandas as pd
import requests

# Where to save parquet files
DATA_DIR = Path("../data").resolve()
DATA_DIR.mkdir(parents=True, exist_ok=True)

# AQS credentials from environment (GitHub-friendly)
AQS_EMAIL = os.getenv("AQS_EMAIL")
AQS_KEY = os.getenv("AQS_KEY")

if not AQS_EMAIL or not AQS_KEY:
    raise RuntimeError(
        "Missing AQS_EMAIL or AQS_KEY environment variables.\n"
        "Set them in your shell, then restart Jupyter.\n"
        "Example (PowerShell):\n"
        '  setx AQS_EMAIL "you@example.com"\n'
        '  setx AQS_KEY   "your_real_aqs_key_here"\n'
    )

BASE_URL = "https://aqs.epa.gov/data/api/dailyData/byState"

STATE_FIPS = "41"     # Oregon
PM25_CODE = "88101"   # PM2.5 - Local Conditions
OZONE_CODE = "44201"  # Ozone

START_YEAR = 2020
END_YEAR = 2024


In [ ]:
def fetch_aqs_daily_by_state(param_code: str, state_fips: str, bdate: str, edate: str) -> pd.DataFrame:
    """
    Call AQS dailyData/byState for a single parameter, state, and date range.

    - param_code: AQS parameter code as string, e.g., "88101" for PM2.5
    - state_fips: 2-digit state FIPS code as string (Oregon = "41")
    - bdate, edate: strings "YYYYMMDD" for start/end dates

    Returns a pandas DataFrame (may be empty).
    """
    params = {
        "email": AQS_EMAIL,
        "key": AQS_KEY,
        "param": param_code,
        "state": state_fips,
        "bdate": bdate,
        "edate": edate,
    }

    r = requests.get(BASE_URL, params=params, timeout=60)
    try:
        r.raise_for_status()
    except requests.HTTPError as e:
        msg = f"HTTP {r.status_code} for {r.url}\nResponse text: {r.text[:300]}"
        raise RuntimeError(msg) from e

    data = r.json()
    if "Data" not in data:
        # API sometimes returns an error structure without raising HTTPError
        raise RuntimeError(f"AQS response missing 'Data' field: {data}")

    df = pd.DataFrame(data["Data"])
    return df


In [ ]:
def fetch_param_over_years(param_code: str, state_fips: str, start_year: int, end_year: int) -> pd.DataFrame:
    """
    Fetch daily AQS data for a given parameter and state over multiple years.

    For each year, we request from YYYY0101 to YYYY1231 (or up to today's date
    for the current year). Results are concatenated into a single DataFrame.
    """
    all_frames = []

    for year in range(start_year, end_year + 1):
        if year == date.today().year:
            # Up to today for the current year
            bdate = f"{year}0101"
            edate = date.today().strftime("%Y%m%d")
        else:
            bdate = f"{year}0101"
            edate = f"{year}1231"

        print(f"Fetching param {param_code} for {state_fips}, {bdate}–{edate} ...", end=" ")

        df_year = fetch_aqs_daily_by_state(param_code, state_fips, bdate, edate)
        print(f"{len(df_year)} rows")

        if not df_year.empty:
            all_frames.append(df_year)

        # Be kind to the API
        time.sleep(1)

    if not all_frames:
        print("No data returned across all years.")
        return pd.DataFrame()

    df_full = pd.concat(all_frames, ignore_index=True)
    # Drop exact duplicates, if any
    df_full = df_full.drop_duplicates()

    return df_full


In [ ]:
df_pm25 = fetch_param_over_years(PM25_CODE, STATE_FIPS, START_YEAR, END_YEAR)
df_ozone = fetch_param_over_years(OZONE_CODE, STATE_FIPS, START_YEAR, END_YEAR)

print("PM2.5 rows:", len(df_pm25))
print("Ozone rows:", len(df_ozone))

pm25_path = DATA_DIR / f"aqs_daily_pm25_{START_YEAR}_{END_YEAR}.parquet"
ozone_path = DATA_DIR / f"aqs_daily_ozone_{START_YEAR}_{END_YEAR}.parquet"

df_pm25.to_parquet(pm25_path, index=False)
df_ozone.to_parquet(ozone_path, index=False)

print("Saved PM2.5 parquet to:", pm25_path)
print("Saved Ozone parquet to:", ozone_path)
